In [1]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import cv2, dlib, sys
import numpy as np

detector = dlib.get_frontal_face_detector()#얼굴 디렉터 모듈 초기화
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
ret_val = cv2.imread('Rocky.jpg', 1)
img = cv2.imread('Rocky.jpg', 1)
img = cv2.resize(img, (int(img.shape[1]), int(img.shape[0])))#동영상 사이즈 너무 커서 resize
ori = img.copy()
    
#detect faces
faces = detector(img)
face = faces[0]

img = cv2.resize(img, (int(img.shape[1]), int(img.shape[0])))#동영상 사이즈 너무 커서 resize
ori = img.copy()
    
#detect faces
faces = detector(img)
face = faces[0]

dlib_shape = predictor(img,face) #img의 face영역 안의 얼굴 특징점 찾기
shape_2d = np.array([[p.x,p.y] for p in dlib_shape.parts()]) #연산 쉽게하기 위함
    
    #compute center and boundaries of face 
    #특징점찾은 이유 : 얼굴의 좌상단, 우하단, 얼굴의 사이즈, 중심 구하기
top_left = np.min(shape_2d, axis=0)
bottom_right = np.max(shape_2d, axis=0)
   
    #우하단에서 좌상단 좌표를 뺀 (x,y 길이의 가장 긴 값)
face_size = int(max(bottom_right - top_left)*2.5)
    
    
    #모든 특징점의 평균 구해서 얼굴 중앙 찾기 : np.mean()
center_x, center_y = np.mean(shape_2d,axis=0).astype(np.int)
    #소수점일수도 있어서 int형으로 변경 : np.astype(np.int) 
    
    #visualize
img = cv2.rectangle(img, pt1=(face.left(), face.top()), pt2=(face.right(), face.bottom()), color=(255,255,255),thickness=2, lineType=cv2.LINE_AA)
    
for s in shape_2d: #얼굴 특징점 68개
    cv2.circle(img, center=tuple(s), radius=1, color=(255,255,255), thickness=2, lineType=cv2.LINE_AA)
    
    #특징점으로 만든 얼굴의 사이즈 -> 원그리기
cv2.circle(img, center=tuple(top_left), radius=1, color=(255,0,0), thickness=2, lineType=cv2.LINE_AA)
cv2.circle(img, center=tuple(bottom_right), radius=1, color=(255,0,0), thickness=2, lineType=cv2.LINE_AA)
cv2.circle(img, center=tuple((center_x, center_y)), radius=1, color=(0,0, 255), thickness=2, lineType=cv2.LINE_AA)

overall_vertices = []
for x in range(0,68):
    coordination = tuple(shape_2d[x])
    overall_vertices.append(coordination + (1,)) # z좌표 값 추가
overall_vertices = tuple(overall_vertices)

colors =((1,0,0),
         (0,1,0),
         (0,0,1),
         (0,1,0))

surfaces=((0,1,1,1),
          (1,1,1,1),
          (1,1,1,1),
          (4,5,1,0),
          (1,5,7,2),
          (4,0,3,6))

vertices = overall_vertices

edges = ((0,1),(0,3),(0,4),
      (2,1),(2,3),(2,7),
      (6,3),(6,4),(6,7),
      (5,1),(5,4),(5,7))

def drawCube():
    glBegin(GL_QUADS)
    for surface in surfaces:
        x = 0
        for vertex in surface:
            glColor3fv(colors[x])
            glVertex3fv(vertices[vertex])
            x += 1
    glEnd()

    glBegin(GL_LINES)
    for edge in edges:
        for vertex in edge:
            glVertex3fv(vertices[vertex])
    glEnd()

def myOpenGL():
    pygame.init()
    display = (800,600)
    pygame.display.set_mode(display,DOUBLEBUF | OPENGL)

    gluPerspective(45,(display[0]/display[1]),0.1,50.0)
    glTranslatef(0.0,0.0,-7)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

        #glRotatef(1,3,1,1)
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        drawCube()
        pygame.display.flip()
        pygame.time.wait(10)

myOpenGL()

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html


GLError: GLError(
	err = 1282,
	description = b'invalid operation',
	baseOperation = glClear,
	cArguments = (16640,)
)